In [48]:
import pandas as pd
import csv

In [49]:
# read original data file from Jordan
raw_df = pd.read_csv("crime_open_database_sample_2018.csv")

In [50]:
# concatenate address but remove all extra delimeters
requests = raw_df.apply(lambda row: (row["city_name"]).replace(",",""), axis=1)
#requests = raw_df.apply(lambda row: (row["agyaddr"]+" "+row["City"]+" "+row["State"] + " "+row["zipcode"]).replace(",",""), axis=1)

In [51]:
# open ucla requests and then paste into geocoder api
# then paste into a new results file
# https://gis.ucla.edu/geocoder

pd.DataFrame(set(requests)).astype(str).to_csv("ucla_requests.csv", header=False, index=False, quoting=csv.QUOTE_NONE, escapechar=" ")

In [52]:
# read lat lon results from UCLA Geocode API
df1 = pd.read_csv("ucla_result.csv")

# remove extra characters and join lat lons to original data file
df1["ADDRESS"] = df1["ADDRESS"].str.replace("'","").str.replace(" ","")
df1 = df1.set_index("ADDRESS")

# create a dictionary that maps addresses to lat or lon
lat_lon_lookup = df1.to_dict()

In [53]:
lats = []
lons = []

# get all lat lons and add the columns to original data file
for row in requests:
    lat = lat_lon_lookup["latitude"][row.replace(" ", "")]
    lon = lat_lon_lookup["longitude"][row.replace(" ", "")]
    lats.append(lat)
    lons.append(lon)
raw_df["latitude"] = lats
raw_df["longitude"] = lons

KeyError: 'latitude'

In [ ]:
raw_df.head()

In [ ]:
import urllib
import json
import requests
import pandas as pd
import csv

# create empty dictionary mapping (lat, lon) to fips code
fips_lookup = {}

bad_lat_lons = set()
# loop through all rows in csv

# stores corresponding fips codes in a list so that we can just add them as a new column afterwards
fips_codes = []
for ind, row in raw_df.iterrows():
    # From dataset, get lat lon from columns for each row
    lat = row["latitude"]
    lon = row["longitude"]
    lookup_key = f"({lat}, {lon})"
    # Check if (lat, lon) is in dictionary
    if lookup_key in fips_lookup:
        # use that stored value if so
        fips_code = fips_lookup[lookup_key]
    else:
        # call API to get FIPS code from lat, lon
        # note: point may be using lon, lat instead of lat, lon
        point_request = f"https://geo.fcc.gov/api/census/block/find?latitude={lat}&longitude={lon}&showall=false&format=json"
        with urllib.request.urlopen(point_request) as url:
            data = json.loads(url.read().decode())
        # get everything except last 4 items from fips block code
        if data["Block"]["FIPS"] is None:
            print(f"something went wrong with ({lat}, {lon})")
            fips_code = None
            bad_lat_lons.add((lat, lon, row["agyaddr"]))
        else:
            fips_code = data["Block"]["FIPS"][:-4]
        fips_lookup[lookup_key] = fips_code
    # append fips_code to end of csv
    fips_codes.append(fips_code)


print("bad lat lons:")
print(bad_lat_lons)
bad_lat_lon_df = pd.DataFrame(bad_lat_lons, columns=["latitude", "longitude", "agyaddr"])
bad_lat_lon_df.to_csv("bad_lat_lons.csv")

# store FIPS codes in raw dataframe
raw_df['FIPS_code'] = fips_codes

# drop row because there are only 130 /14000 rows, negligible
raw_df = raw_df.dropna(subset=['FIPS_code'])

In [ ]:
raw_df.head()